# Import libary

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def initializeCentroid(X, K):
    return random.sample(list(X), K)

In [ ]:
def findClosestCentroid(X, centroid):
    K = np.size(centroid, 0)
    index = np.zeros((np.size(X, 0), 1))
    arr = np.empty((np.size(X, 0), 1))
    for i in range (0, K):
            y = centroid[i]
            temp = np.ones((np.size(X, 0), 1))* y
            b = np.power
